In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
# MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

In [ ]:
# X(images), Y(labels)
X = tf.placeholder(tf.float32, [None, 28*28]) # inf. x 784(28*28)
Y = tf.placeholder(tf.int64, [None]) # inf. x 1

In [ ]:
batch_size=100
learning_rate=0.01

In [ ]:
# MLP Model
def model(input_X):
    # 히든 레이어1
    W1 = tf.Variable(tf.truncated_normal([28*28, 128], stddev=0.1))
    b1 = tf.Variable(tf.zeros([128]))
    h1 = tf.nn.tanh(tf.matmul(input_X, W1) + b1)

    # 히든 레이어2
    W2 = tf.Variable(tf.truncated_normal([128, 64], stddev=0.1))
    b2 = tf.Variable(tf.zeros([64]))
    h2 = tf.nn.tanh(tf.matmul(h1, W2) + b2)

    # 출력 레이어 (최종 출력 10개)
    class_num = 10
    W_fc = tf.Variable(tf.truncated_normal([64, class_num], stddev=0.1))
    b_fc = tf.Variable(tf.zeros([class_num]))
    pred = tf.matmul(h2, W_fc) + b_fc

    return pred

In [ ]:
pred = model(X)

In [ ]:
# training 반복 횟수
max_step = 10000

# Cost Function 설계 (Cross Entropy)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=pred)
cost = tf.reduce_mean(cross_entropy)

# Gradient descent Optimizer(학습)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
# Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(1, max_step + 1):
    X_images, X_labels = mnist.train.next_batch(batch_size)
    X_labels = np.where(X_labels)[1]
        
    sess.run(optimizer, feed_dict={X: X_images, Y: X_labels})

    # log
    loss = sess.run(cost, feed_dict={X: X_images, Y: X_labels})
    if epoch % 500 == 0 or epoch == 0:
        print("epoch: %05d,"%(epoch), "loss:", loss)

print("Finished! (loss : " + str(loss) + ")")

In [ ]:
# test_images, test_labels = mnist.test.next_batch(16)
# test_labels = np.where(test_labels)[1]
    
nCnt = 0
tmp_sum = []
test_input = 100
for i in range(1):
    x_test, test_label = mnist.test.next_batch(test_input)
    arr_data = sess.run(pred, feed_dict={X: x_test})

    pred_val = tf.argmax(arr_data, 1)
    real_val = np.where(test_label)[1]
    
    for ib in range(test_input):
        tmp_pred = pred_val.eval(session=sess)
        tmp_real = real_val
        
        if tmp_pred[ib] == tmp_real[ib]:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => True")
            nCnt += 1
        else:
            print("예측값: " + str(tmp_pred[ib]) + " / 실제값: " + str(tmp_real[ib]) + " => False")
        tmp_sum.append(nCnt)
        nCnt = 0
tmp_sum = np.array(tmp_sum)

print("Accuracy:", (tmp_sum.mean()))